In [33]:
# Core python libraries
import csv
import re
import math
import os
import time

# External libraries.
import nltk
import numpy
import pandas
import scipy
import sklearn
import tempfile

# Tensorflow and related.
import tensorflow as tf

# Plotting
import matplotlib.pyplot as plt

import imp

import tf_utils
tf_utils = imp.reload(tf_utils)

# So you know when this code block finishes.
print ("Done")

Done


In [2]:
#TODO(Max): make a library that does all this preprocessing
data = pandas.read_csv("data/train.csv") # Can read a subset. First nrows of the total.
LABEL_COLUMN = "SalaryNormalized"

TEST_SIZE = .3


print(len(data))

171482


In [3]:
# sklearn.cross_validation has been replaced with model_selection.
X_train_index, X_test_index, Y_train, Y_test = sklearn.model_selection.train_test_split(
    data.index, data[LABEL_COLUMN], test_size=TEST_SIZE, random_state=42)

# Keep train and test as pandas dataframes.
X_train = data.iloc[X_train_index]
X_test = data.iloc[X_test_index]

y_train = numpy.array(X_train[LABEL_COLUMN].astype(numpy.float32))
y_test = numpy.array(X_test[LABEL_COLUMN].astype(numpy.float32))

print ("Done")

Done


In [4]:
# Normalizing input values to mean 0, standard deviation 1.
train_mean, train_std = y_train.mean(), y_train.std()

# Normalize input labels and expectations
y_train = tf_utils.normalize_input(y_train, train_mean, train_std)
y_test = tf_utils.normalize_input(y_test, train_mean, train_std)


In [5]:
print(train_std)
print(y_test)

17698.3
[ 0.89645004 -0.66550416  3.22435188 ..., -0.29010177  0.89645004
  0.95295256]


In [6]:
target_column = "CompleteJobListingStemmed"

x_train_text = X_train[target_column]
x_test_text = X_test[target_column]

In [16]:
min_word_frequency = 59

sklearn_bow = True # Otherwise we use Tensorflow BOW.

if sklearn_bow:
    # Gives 7600 unique words with min frequency 75 and stopwords removed.
    # 7797 if we leave stopwords in.

    count_vect = sklearn.feature_extraction.text.CountVectorizer(
        min_df=min_word_frequency, decode_error='ignore')

    # Transform BOW test set.
    x_train = count_vect.fit_transform(x_train_text)
    x_test = count_vect.transform(x_test_text)
    n_words = x_train.shape[1]
    max_document_length = n_words

else:
    # Look here for a tutorial https://medium.com/@ilblackdragon/tensorflow-text-classification-615198df9231
    # Gives 8712 unique words with min word frequency of 75
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
        max_document_length, min_frequency=min_word_frequency)
    x_train = numpy.array(list(vocab_processor.fit_transform(x_train_text)))
    x_test = numpy.array(list(vocab_processor.transform(x_test_text)))
    n_words = len(vocab_processor.vocabulary_)
    max_document_length = max([len(x.split(" ")) for x in x_train_text])
    
print("x_train shape: %s, %s" % x_train.shape)
print("x_test shape: %s, %s" % x_test.shape)
print("Total distinct words: %d" % n_words)
print("Number of training rows: %d" % len(x_train))

x_train shape: 120037, 8762
x_test shape: 51445, 8762
Total distinct words: 8762


In [18]:
# Define data nput functions. These have to be done here as the 
# functions take no arguments. Also the dict labels are specific to 
# the model.
max_document_length = n_words

def input_fn():
    for batch_input, batch_labels in tf_utils.generate_batch(
        batch_size, number_of_batches, max_document_length,
                                                    x_train, y_train):
        yield {x_data: batch_input, y_target: batch_labels}

def eval_input_fn():
    for batch_input, batch_labels in tf_utils.generate_batch(
        batch_size, number_of_validation_batches, max_document_length,
                                                    x_test, y_test):
        yield {x_data: batch_input, y_target: batch_labels}


In [49]:
graph = tf.Graph()

sess = tf.Session(graph=graph)

learning_rate = .01

l1_regularization_coef = 0.01
l2_regularization_coef = 0.01

# if not RMSE then MAE.
HUBER = False
RMSE = True

#Implement this by batch.
batch_size = 1024

"""
def huber_loss(labels, predictions, delta=1.0):
 residual = tf.abs(predictions - labels)
 condition = tf.less(residual, delta)
 small_res = 0.5 * tf.square(residual)
 large_res = delta * residual - 0.5 * tf.square(delta)
 return tf.reduce_mean(tf.where(condition, small_res, large_res))
"""
with graph.as_default():

    with tf.name_scope("data"):
        # Initialize data placeholders
        x_data = tf.placeholder(shape=[batch_size, max_document_length], dtype=tf.float32)
        y_target = tf.placeholder(shape=[batch_size, 1], dtype=tf.float32, name="labels")
    """
    with tf.name_scope("bag_of_words"):
        with tf.device('/cpu:0'):
            identity_mat = tf.diag(tf.ones(shape=[n_words]))
            # This is also an embedding_lookup_sparse function that would create a sparse tensor.
            x_embed = tf.nn.embedding_lookup(identity_mat, x_data)
            # Collapse across the document length dimension. Leaving only the number of tokens.
            bag_of_words = tf.reduce_sum(x_embed, 1)

            print("Woohoo this looks like my bag of words shape :D %s" % bag_of_words.shape)
    """
    # Create variables for regression
    A = tf.Variable(tf.zeros(shape=[n_words, 1]))
    b = tf.Variable(tf.zeros(shape=[1]))

    # A = tf.Variable(tf.random_normal(shape=[vocab_size, 1], stddev=.05))
    # Intercept term. A scalar.
    # b = tf.Variable(tf.random_normal(shape=[1], stddev=.05))

    # Model output
    with tf.device('/gpu:0'):

        product = tf.matmul(x_data, A)
        model_output = tf.add(product, b, name="predictions")

    with tf.name_scope("loss"):

        # TODO(Max) try root mean square error. Should this be merely mean square error?
        if HUBER:
            loss = huber_loss(y_target, model_output)
        elif RMSE:
            loss = tf.sqrt(tf.reduce_mean(tf.pow(tf.subtract(y_target, model_output), 2)))
        else:
            # Mean Absolute Error.
            loss = tf.reduce_mean(tf.abs(tf.subtract(y_target, model_output)))

        # If we are using regularization.
        if l1_regularization_coef > 0 or l2_regularization_coef > 0:
            l1_regularizer = tf.contrib.layers.l1_regularizer(scale=l1_regularization_coef)
            l2_regularizer = tf.contrib.layers.l2_regularizer(scale=l2_regularization_coef)

            weights = tf.trainable_variables() # all vars of your graph
            regularization_penalty = tf.add(tf.contrib.layers.apply_regularization(l1_regularizer, weights),
                                             tf.contrib.layers.apply_regularization(l2_regularizer, weights))
            loss = loss + regularization_penalty # this loss needs to be minimized

    with tf.name_scope("reporting"):
        # Converts back to original, salary scale.
        error_salary_scale = tf.multiply(tf.subtract(y_target, model_output), train_std)

        mean_absolute_error_salary_scale = tf.reduce_mean(
            tf.abs(error_salary_scale))

        # Log for tensorboard
        training_summary = tf.summary.scalar('train_loss', loss)
        validation_summary = tf.summary.scalar('validation_loss', loss)
        # Add mean absolute errors.
        training_mae = tf.summary.scalar('train_mae', mean_absolute_error_salary_scale)
        validation_mae = tf.summary.scalar('validation_mae', mean_absolute_error_salary_scale)

    global_step = tf.Variable(0, name='global_step', trainable=False)

    # Declare optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)

    
    # This one seems to work better :D
    train_op = tf.contrib.layers.optimize_loss(      
     loss, tf.contrib.framework.get_global_step(),      
     optimizer='Adam'
        , learning_rate=learning_rate, clip_gradients=1.0)


In [52]:
# Number of
number_of_epochs = 50
batches_per_epoch = len(x_train_text)/batch_size
number_of_batches = math.ceil(len(x_train_text)/batch_size)
number_of_validation_batches = number_of_batches

logs_dir = "logs/bagofwords"

checkpoint_frequency = 500

reporting_frequency = 3

total_loss = 0
validation_loss = 0
validation_batch_average_window = 100

ckpt = None
ckpt = tf.train.get_checkpoint_state(logs_dir)  

In [21]:
QUEUE_LIMIT = 500000

# Try me out
#batch_size = 256


with graph.as_default():
    queue = tf.RandomShuffleQueue(
        capacity=QUEUE_LIMIT, min_after_dequeue=batch_size,
        dtypes=[tf.float32, tf.float32],
        shapes = [(n_words,), (1,)],
        seed=1999)

    with tf.name_scope("preprocess_data"):
        with tf.device('/cpu:0'):

            # Initialize data placeholders
            x_data = tf.placeholder(shape=[batch_size, max_document_length], dtype=tf.int32)
            y_target = tf.placeholder(shape=[batch_size, 1], dtype=tf.float32, name="labels")
            #with tf.device('/cpu:0'):
            identity_mat = tf.diag(tf.ones(shape=[vocab_size]))
            # This is also an embedding_lookup_sparse function that would create a sparse tensor.
            x_embed = tf.nn.embedding_lookup(identity_mat, x_data)
            # Collapse across the document length dimension. Leaving only the number of tokens.
            example = tf.reduce_sum(x_embed, 1)

            enqueue_op = queue.enqueue_many((example, y_target))

        # Dequeue our data. Use dequeue_many(batch_size) later.
        dequeue_op = queue.dequeue_many(batch_size)

# Create 8 threads.
#qr = tf.train.QueueRunner(queue, [enqueue_op] * 8)
# Launch the graph.
with tf.Session(graph=graph) as session:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)


#sess = tf.Session()
    #enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    for step in range(10): # do to 100 iterations
        #if coord.should_stop():
        #    break
        #print(feed_dict)
        start_time = time.time()
        feed_dict = input_fn()
        #print(feed_dict)
        thing = session.run([enqueue_op], feed_dict=next(feed_dict))
        duration = time.time() - start_time
        print("Batch %s Enqueue Duration: %.4f" % (step +1, duration))
    coord.request_stop()
    coord.join(threads)

<generator object input_fn at 0x7f4a9ca09a98>
Batch 1 Enqueue Duration: 7.1514
<generator object input_fn at 0x7f4a9e63a5c8>


KeyboardInterrupt: 

In [18]:
with tf.Session(graph=graph) as session:
    print(tf.get_default_graph().as_graph_def())

node {
  name: "data/Placeholder"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 256
        }
        dim {
          size: 1368
        }
      }
    }
  }
}
node {
  name: "data/labels"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 256
        }
        dim {
          size: 1
        }
      }
    }
  }
}
node {
  name: "bag_of_words/ones"
  op: "Const"
  device: "/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 8712
          }
        }
        float_val: 1.0
      }
    }
  }
}
node {
  name: "bag_of_words/Diag"
  op: "Diag"
  input: "bag_of_words/ones"
  device: "/dev

In [ ]:
with tf.Session(graph=graph) as session:
    #enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    for step in range(10): # do to 100 iterations
        #if coord.should_stop():
        #    break
        #print(feed_dict)
        data, label = session.run([dequeue_op])
        print(data)
        print(label)

In [50]:
import tf_utils
tf_utils = imp.reload(tf_utils)

print(type(x_train))
for a, b in input_fn():
    print(a)
    print(b)
    break

<class 'scipy.sparse.csr.csr_matrix'>
Tensor("data/Placeholder:0", shape=(1024, 8762), dtype=float32)
Tensor("data/labels:0", shape=(1024, 1), dtype=float32)


In [53]:
# Where to save this model.
model_path = os.path.join(logs_dir, time.strftime("%Y-%m-%d-%H-%M-%S"))

import tf_utils
tf_utils = imp.reload(tf_utils)

if ckpt:
    restore_from_model = True
    print("Restore from model!")
else:
    restore_from_model = False

with tf.Session(graph=graph, config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True)) as session:
    summaries = tf.summary.merge_all()                                                       
    writer = tf.summary.FileWriter(model_path,
        graph=session.graph)                  

    saver = tf.train.Saver(tf.global_variables()) 
                                                                                                  
    session.run(tf.global_variables_initializer())
    
    
    if restore_from_model:
        saver.restore(session, ckpt.model_checkpoint_path)
        
    global_step = 0
        
    for epoch in range(number_of_epochs):
        eval_feed_generator = eval_input_fn()
        # Shuffle the training input
        x_train, y_train = tf_utils.unison_shuffled_copies(x_train, y_train)

        for step, feed_dict in enumerate(input_fn()):
            global_step += 1
            start_time = time.time()

            _, temp_loss = session.run([train_op, loss], feed_dict=feed_dict)
            
            duration = time.time() - start_time
            
            if global_step % reporting_frequency == 0:
                print("Epoch: %s Iteration: %s : Loss: %.4f (%.3f sec)" % 
                      (epoch, (step + 1), temp_loss, duration))

                train_summ, train_mae = session.run([training_summary, training_mae], feed_dict=feed_dict)
                writer.add_summary(train_summ, global_step)
                writer.add_summary(train_mae, global_step)
                _, valid_summ, valid_mae = session.run(
                        [loss, validation_summary, validation_mae],
                        feed_dict=next(eval_feed_generator))
                writer.add_summary(valid_summ, global_step)
                writer.add_summary(valid_mae, global_step)

                writer.flush()                

            if (global_step % checkpoint_frequency == 0) or (global_step == number_of_batches):
                print("Saving at epoch %s step: %s" % (epoch, step + 1))
                saver.save(session, model_path, global_step=tf.contrib.framework.get_global_step())

    



Restore from model!
INFO:tensorflow:Restoring parameters from logs/bagofwords/2017-06-27-20-43-59-32819
Epoch: 0 Iteration: 5 : Loss: 0.9814 (0.021 sec)
Epoch: 0 Iteration: 10 : Loss: 1.0438 (0.018 sec)
Epoch: 0 Iteration: 15 : Loss: 0.9693 (0.020 sec)
Epoch: 0 Iteration: 20 : Loss: 1.0008 (0.018 sec)
Epoch: 0 Iteration: 25 : Loss: 1.0250 (0.018 sec)
Epoch: 0 Iteration: 30 : Loss: 1.0122 (0.018 sec)
Epoch: 0 Iteration: 35 : Loss: 0.9932 (0.018 sec)
Epoch: 0 Iteration: 40 : Loss: 1.0112 (0.018 sec)
Epoch: 0 Iteration: 45 : Loss: 0.9558 (0.018 sec)
Epoch: 0 Iteration: 50 : Loss: 0.9573 (0.018 sec)
Epoch: 0 Iteration: 55 : Loss: 0.9766 (0.018 sec)
Epoch: 0 Iteration: 60 : Loss: 1.0086 (0.017 sec)
Epoch: 0 Iteration: 65 : Loss: 1.0192 (0.018 sec)
Epoch: 0 Iteration: 70 : Loss: 0.9899 (0.018 sec)
Epoch: 0 Iteration: 75 : Loss: 0.9895 (0.018 sec)
Epoch: 0 Iteration: 80 : Loss: 0.9920 (0.017 sec)
Epoch: 0 Iteration: 85 : Loss: 0.9983 (0.018 sec)
Epoch: 0 Iteration: 90 : Loss: 1.0129 (0.018 se

In [54]:
# How much data to evaluate
#number_of_batches = len(x_train_text)/batch_size
number_of_validation_batches = math.floor(len(x_test_text)/batch_size)
ckpt = tf.train.get_checkpoint_state(logs_dir)                                     

predictions = []
# Write TF Model Evaluation Code here. 
with tf.Session(graph=graph, config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True)) as session:
    
    saver = tf.train.Saver(tf.global_variables()) 
                                                                                                  
    session.run(tf.global_variables_initializer())
    
    saver.restore(session, ckpt.model_checkpoint_path)

    total_loss = 0
    for step, eval_feed_dict in enumerate(eval_input_fn()):    
        valid_loss, mae = session.run(
            [loss, mean_absolute_error_salary_scale], feed_dict=eval_feed_dict)
        #print("Batch: %d Mean: %.4f" % (step, mae))
        total_loss += mae
    print ("Mean Absolute Error Loss over all %s observations = %s" % ((step + 1) * batch_size, total_loss / (step + 1)))

INFO:tensorflow:Restoring parameters from logs/bagofwords/2017-06-27-20-46-01-38319
Mean Absolute Error Loss over all 51200 observations = 9475.84244141


In [114]:
%matplotlib inline

def print_results(model):
    y_vals = [value for value in model.predict(input_fn=eval_input_fn)]
    x_vals = X_test[LABEL_COLUMN]


    plt.scatter(x_vals, y_vals)
    plt.show()

    print(x_vals-y_vals)
    #print(sorted([set(y_vals)]))

    print("Mean absolute error: %s" % 
          mean_absolute_error_salary_scale(y_vals, x_vals))
    
    
print_results(m)

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.


NotFittedError: Couldn't find trained model at /tmp/tmpc4vvis3j.